In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# SETUP
import all needed libraries and define the root directory

In [3]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt # plotting
%matplotlib inline
import time
import os
import pandas as pd
import shutil
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# library used to process audio 
import librosa
from librosa import display
import IPython.display as ipd
import json
import random
plt.figure(figsize=(14,5))
#DATAPATH = './archive'
#dir_list = os.listdir(DATAPATH)
#print(dir_list)
#print(os.listdir(os.path.join(DATAPATH,dir_list[0])))

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPool2D, Dropout, Flatten, Dense, Activation, BatchNormalization, Lambda
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
ROOT = "/kaggle/input/ravdess-emotional-speech-audio"

## Code used to check the GPU availability

In [34]:
from tensorflow.python.client import device_lib
tf.config.experimental.list_physical_devices('GPU')
print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

## define functions and take a look at the audio

In [35]:
testpath = os.path.join('/kaggle/input/ravdess-emotional-speech-audio', "Actor_01/03-01-01-01-01-01-01.wav")
def generate_sound(path):
    ipd.Audio(path)
def generate_waveplot(path, title):
    plt.title('Waveplot for audio {}'.format(title), size=15)
    x,fs = librosa.load(testpath)
    librosa.display.waveplot(x, sr=fs)
    plt.xlabel("Time")
    plt.ylabel("Amplitude")
    plt.show()

def generate_spectrogram(path, title):
    x,fs = librosa.load(testpath)
    x = librosa.stft(x)
    x = librosa.amplitude_to_db(abs(x))
    plt.title('Spectrogram for audio {}'.format(title), size=15)
    librosa.display.specshow(x, sr=fs, x_axis='time', y_axis='hz')  
    plt.colorbar()
generate_sound(testpath)
generate_waveplot(testpath, "sample sound")
generate_spectrogram(testpath,"sample sound")

In [6]:
data, rate = librosa.load(testpath)

# Emotion Representatin

https://www.youtube.com/watch?v=26_qiXEa8lw </br>

* three Dimensional
        Valence (negative - positive)
        Activation (low - high)
        Dominance (Dominated - dominant)

* Two Dimensional
        arousal
        valence
* features
        zero-crossing rate
        chroma_stft
        chroma_cqt
        chroma_cens
        mel-frequency cepstral coefficient
        root mean square value
        melspectogram

https://academic.microsoft.com/topic/2776860830/publication/search?q=Zero-crossing%20rate&qe=And(Composite(F.FId%253D2776860830)%252CTy%253D%270%27)&f=&orderBy=0 </br>

is the rate at which a signal changes from positive to negative or from negative to positive. is a key feature to classify percussive sounds

In [38]:
test_zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
len(test_zcr)

In [39]:
stft = np.abs(librosa.stft(data))
test_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=rate).T, axis=0)
len(test_stft)

https://www.youtube.com/watch?v=4_SH2nfbQZ8 </br>
https://en.wikipedia.org/wiki/Mel-frequency_cepstrum </br>

* MFCCs advantages

Describe the "large" structures of the spectrum

ignore fine spectral structures, really don't care on the pitch, etc.

work wll in speech and music processing
* MFCCs are commonly used as features in speech recognition systems, such as the systems which can automatically recognize numbers spoken into a telephone

In [40]:
test_mfcc = np.mean(librosa.feature.mfcc(y=data, sr=rate).T, axis=0)
len(test_mfcc)

In [41]:
test_rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
len(test_rms)

In [42]:
test_mels = np.mean(librosa.feature.melspectrogram(y=data, sr=rate).T, axis=0)
len(test_mels)

In [43]:
cqt = np.mean(librosa.feature.chroma_cqt(y=data, sr=rate).T, axis=0)
len(cqt)

In [44]:
cens = np.mean(librosa.feature.chroma_cens(y=data, sr=rate).T, axis=0)
len(cens)

# Data Augmentation

* noise
* time shifting
* time stretching
* pitch shifting

Data augmentation is a method for generating synthetic data


Sound wave has following characteristics: Pitch, Loudness, Quality. We need to alter our samples around these characteristics in such a way that they only differ by small factor from original sample.

In [37]:
def noise(wav, p):
    return wav + p*np.random.normal(0,1,len(wav))
def time_shifting(wav,p):
    return np.roll(wav,p)
def time_stretching(wav,p):
    return librosa.effects.time_stretch(wav,p)
def pitch_shifting(wav,sr, p):
    return librosa.effects.pitch_shift(wav,sr,n_steps=p)

In [8]:
librosa.display.waveplot(pitch_shifting(data,rate,-5), sr=rate)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

In [9]:
librosa.display.waveplot(noise(data,0.004), sr=rate)
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.show()

In [10]:
plt.figure(figsize=(12, 10), dpi=80)
plt.subplot(2,2,1)
librosa.display.waveplot(noise(data,0.004), sr=rate)
plt.title("noise")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.subplot(2,2,2)
librosa.display.waveplot(time_shifting(data,int(rate*100)), sr=rate)
plt.title("time shifting")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.subplot(2,2,3)
librosa.display.waveplot(time_stretching(data,0.5), sr=rate)
plt.title("time stretching")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.subplot(2,2,4)
librosa.display.waveplot(pitch_shifting(data,rate,-5), sr=rate)
plt.title("pitch shifting")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.savefig("data_augmentations.png")
plt.show()

# Extract Data

In [11]:
def files_info_extractor(path):
    dir_list = os.listdir(path)
    y = []
    X = []
    for dir in dir_list:
        files = os.listdir(os.path.join(ROOT,dir))
        for filename in files:
        #print(os.path.join(dirname, filename))
            l = len(filename)
            if (filename[l-3:l] == "wav"):
                X.append([os.path.join(ROOT,dir,filename),
                      int(filename[0:2]),
                     int(filename[3:5]),
                     int(filename[6:8]),
                     int(filename[9:11]),
                    int(filename[12:14]),
                    int(filename[15:17]),
                    int(filename[18:20])])
    return X
info = files_info_extractor(ROOT)
info_df = pd.DataFrame(info, columns =["Url", 
                               'Modality', 
                               'Vocal channel',
                              'Emotion',
                              'Emotional intensity',
                              'Statement',
                              'Repetition',
                              'Actor'])

In [12]:
info_df

In [13]:
plt.title('Count of Emotions', size=16)
sns.countplot(info_df["Emotion"])
plt.ylabel('Count', size=12)
plt.xlabel('Emotions', size=12)
sns.despine(top=True, right=True, left=False, bottom=False)
plt.show()

# Define functions to generate features

In [14]:
def generate_zcr(data,rate):
    return np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
def generate_stft(data,rate):
    stft = np.abs(librosa.stft(data))
    return np.mean(librosa.feature.chroma_stft(S=stft, sr=rate).T, axis=0)
def generate_mfcc(data,rate):
    return np.mean(librosa.feature.mfcc(y=data, sr=rate).T, axis=0)

def generate_rms(data, rate):
    return np.mean(librosa.feature.rms(y=data).T, axis=0)
def generate_mels(data, rate):
    return np.mean(librosa.feature.melspectrogram(y=data, sr=rate).T, axis=0)
def generate_cqt(data, rate):
    return np.mean(librosa.feature.chroma_cqt(y=data,sr=rate).T,axis=0)
def generate_cens(data, rate):
    return np.mean(librosa.feature.chroma_cens(y=data,sr=rate).T, axis=0)

In [15]:
def featuresGenerator(data,rate,paras):
    features = np.array([])
    for para in paras:
        if (para == "zcr"):
            features = np.hstack((features,generate_zcr(data, rate)))
        elif (para=="stft"):
            features = np.hstack((features,generate_stft(data, rate)))
        elif (para=="mfcc"):
            features = np.hstack((features,generate_mfcc(data, rate)))
        elif (para == "rms"):
            featurse = np.hstack((features,generate_rms(data, rate)))
        elif (para == "mels"):
            features = np.hstack((features,generate_mels(data, rate)))
        elif (para == "cqt"):
            features = np.hstack((features,generate_cqt(data, rate)))
        elif (para == "cens"):
            features = np.hstack((features,generate_cens(data,rate)))
        else:
            print("wrong features")
    return features


def extract_feature(path,paras, augs):
    data, rate = librosa.load(path)
    if (len(augs)==0):
        return np.array([featuresGenerator(data,rate,paras)])
    result = np.array(featuresGenerator(data,rate,paras))
    for aug in augs:
        if (aug == "noise"):
            result = np.vstack((result,featuresGenerator(noise(data,0.004),rate,paras)))
        elif (aug == "time_shift"):
            result = np.vstack((result, featuresGenerator(time_shifting(data,int(rate/10)),rate,paras)))
        elif (aug == "time_stretch"):
            result = np.vstack((result, featuresGenerator(time_stretching(data,0.5),rate,paras)))
        elif (aug == "pitch_shift"):
            result = np.vstack((result, featuresGenerator(pitch_shifting(data,rate,-5),rate,paras)))
        else:
            print("wrong augmentation")
    return result

def get_features(paras,augs):
    x_train =[]
    y_train = []
    x_val = []
    y_val = []
    x_test= []
    y_test = []
    train,rest = train_test_split(info, random_state=0, test_size=0.4)
    val, test = train_test_split(rest,random_state=0,test_size=0.5)
    for i in range(len(train)):
        path = train[i][0]
        features = extract_feature(path,paras,augs)
        for f in features:
            x_train.append(f)
            y_train.append(train[i][3])
    for i in range(len(val)):
        path = val[i][0]
        features = extract_feature(path,paras,[])
        for f in features:
            x_val.append(f)
            y_val.append(val[i][3])
    for i in range(len(test)):
        path = test[i][0]
        features = extract_feature(path,paras,[])
        for f in features:
            x_test.append(f)
            y_test.append(test[i][3])
    print("train contains "+str(len(train))+" audios")
    print("train contains "+str(len(val))+" audios")
    print("train contains "+str(len(test))+" audios")
    return x_train, y_train, x_val, y_val, x_test, y_test

# Define callback list here

In [24]:
#os.system('mkdir "saved_models"')
#filepath = "saved_models/weight-model-improvement-{epoch:02d}-{val_accuracy:.2f}-"+time.strftime("%Y%m%d-%H%M%S")+".h5"
#checkpoint = ModelCheckpoint(filepath, monitor="val_accuracy",verbose=1,save_best_only=True, model="max", save_weights_only=False)
earlystopping = EarlyStopping(monitor="val_loss",patience=20,verbose=1,restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.1, patience=5, verbose=0,
    mode='auto', min_delta=0.0001, cooldown=0, min_lr=0,
)
log_csv = CSVLogger("my_logs.csv",separator=',', append=False)
callback_list = [earlystopping]
callback_list_with_reduction = [earlystopping,log_csv,reduce_lr]

# Build baseline

In [18]:
def build_baseline(x_train):
    model=Sequential()
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(pd.DataFrame(x_train).shape[1], 1)))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    model.compile(optimizer = Adam(learning_rate=0.001) , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    return model

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()

In [ ]:
model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
        
history=model.fit(x_train, y_train, batch_size=100, epochs=200, validation_data=(x_val, y_val))
loss_history = pd.DataFrame(history.history)
loss_history[['accuracy','val_accuracy']].plot()

In [ ]:
test_performance = []
test_performance.append(model.evaluate(x_test,y_test))
test_performance

In [ ]:
json.dump(history.history,open("/kaggle/working/baseline_history",'w'))

In [ ]:
model.save("baseline.h5")

In [ ]:
baseline_history = json.load(open("/kaggle/working/baseline_history","r"))
plt.plot(baseline_history["accuracy"],label="accuracy")
plt.plot(baseline_history["val_accuracy"],label="val_accuracy")
plt.legend()
plt.savefig("baseline_accuracy.png")
plt.show()

In [ ]:
plt.plot(baseline_history["loss"],label="loss")
plt.plot(baseline_history["val_loss"],label="val_loss")
plt.legend()
plt.ylim((0,5))
plt.savefig("baseline_loss.png")
plt.show()

In [ ]:
#baseline_model = build_baseline(x_train)
#baseline_model.load_weights("baseline_weights.h5")
model.evaluate(x_test,y_test)

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model,to_file="baseline_model.png", show_shapes=True, show_layer_names=True)

# Analyze Data augmentation Techniques

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],["noise"])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
history=model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(history.history,open("/kaggle/working/noise_history",'w'))
model.save("noise.h5")
test_performance.append(model.evaluate(x_test,y_test))
test_performance

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],["noise","time_shift"])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
history=model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(history.history,open("/kaggle/working/noise_timeshift_history",'w'))
model.save("noise_timeshift.h5")
test_performance.append(model.evaluate(x_test,y_test))
test_performance

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],["noise","time_shift","time_stretch"])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
history=model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(history.history,open("/kaggle/working/noise_timeshift_timestretch_history",'w'))
model.save("noise_timeshift_timestretch.h5")
test_performance.append(model.evaluate(x_test,y_test))
test_performance

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],["noise","time_shift","time_stretch","pitch_shift"])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
history=model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(history.history,open("/kaggle/working/noise_timeshift_timestretch_pitchshift_history",'w'))
model.save_weights("noise_timeshift_timestretch_pitchshift_weights.h5")

In [ ]:
baseline_history = json.load(open("/kaggle/working/baseline_history","r"))
noise_history = json.load(open("/kaggle/working/noise_history","r"))
noise_timeshift_history = json.load(open("/kaggle/working/noise_timeshift_history","r"))
noise_timeshift_timestretch_history = json.load(open("/kaggle/working/noise_timeshift_timestretch_history","r"))
noise_timeshift_timestretch_pitchshift_history = json.load(open("/kaggle/working/noise_timeshift_timestretch_pitchshift_history","r"))
plt.plot(baseline_history["val_accuracy"],label="baseline")
plt.plot(noise_history["val_accuracy"],label="noise")
plt.plot(noise_timeshift_history["val_accuracy"],label="noise timeshift")
plt.plot(noise_timeshift_timestretch_history["val_accuracy"],label="noise timeshift timestretch")
plt.plot(noise_timeshift_timestretch_pitchshift_history["val_accuracy"],label="noise timeshift timestretch pitchshift")
plt.legend()
plt.savefig("data_augs_acc.png")
plt.show()

In [ ]:
plt.plot(baseline_history["val_loss"],label="baseline")
plt.plot(noise_history["val_loss"],label="noise")
plt.plot(noise_timeshift_history["val_loss"],label="noise timeshift")
plt.plot(noise_timeshift_timestretch_history["val_loss"],label="noise timeshift timestretch")
plt.plot(noise_timeshift_timestretch_pitchshift_history["val_loss"],label="noise timeshift timestretch pitchshift")
plt.legend()
plt.ylim((0,5))
plt.savefig("data_augs_loss.png")
plt.show()

# Analyze Features selection
* zero-crossing rate
* mel-frequency cepstral coefficient
* melspectogram
* chroma_stft
* chroma_cqt
* chroma_cens


In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
mfcc_model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
mfcc_history=mfcc_model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(mfcc_history.history,open("/kaggle/working/mfcc_history",'w'))

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mels"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
mels_model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
mels_history=mels_model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(mels_history.history,open("/kaggle/working/mels_history",'w'))

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mels","mfcc"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
mels_mfcc_model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
mels_mfcc_history=mels_mfcc_model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(mels_mfcc_history.history,open("/kaggle/working/mels_mfcc_history",'w'))

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc","zcr"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
mfcc_zcr_model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
mfcc_zcr_history=mfcc_zcr_model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(mfcc_zcr_history.history,open("/kaggle/working/mfcc_zcr_history",'w'))

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc","cens"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
mfcc_cens_model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
mfcc_cens_history=mfcc_cens_model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(mfcc_cens_history.history,open("/kaggle/working/mfcc_cens_history",'w'))

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc","cens","cqt","stft"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
mfcc_chroma_model = build_baseline(x_train_2d)
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)
mfcc_chroma_history=mfcc_chroma_model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val))

In [ ]:
json.dump(mfcc_chroma_history.history,open("/kaggle/working/mfcc_chroma_history",'w'))

In [ ]:
mfcc_history = json.load(open("/kaggle/working/mfcc_history","r"))
mels_history = json.load(open("/kaggle/working/mels_history","r"))
mels_mfcc_history = json.load(open("/kaggle/working/mels_mfcc_history","r"))
mfcc_zcr_history = json.load(open("/kaggle/working/mfcc_zcr_history","r"))
mfcc_cens_history = json.load(open("/kaggle/working/mfcc_cens_history","r"))
mfcc_chroma_history = json.load(open("/kaggle/working/mfcc_chroma_history","r"))
plt.plot(mfcc_history["val_accuracy"],label="mfcc")
plt.plot(mels_history["val_accuracy"],label="mels")
plt.plot(mels_mfcc_history["val_accuracy"],label="mfcc mels")
plt.plot(mfcc_zcr_history["val_accuracy"],label="mfcc zcr")
plt.plot(mfcc_cens_history["val_accuracy"],label="mfcc cens")
plt.plot(mfcc_chroma_history["val_accuracy"],label="mfcc chroma")
plt.legend()
plt.savefig("features_acc.png")
plt.show()

In [ ]:
plt.plot(mfcc_history["val_loss"],label="mfcc")
plt.plot(mels_history["val_loss"],label="mels")
plt.plot(mels_mfcc_history["val_loss"],label="mfcc mels")
plt.plot(mfcc_zcr_history["val_loss"],label="mfcc zcr")
plt.plot(mfcc_cens_history["val_loss"],label="mfcc cens")
plt.plot(mfcc_chroma_history["val_loss"],label="mfcc chroma")
plt.legend()
plt.ylim((0,15))
plt.savefig("features_loss.png")
plt.show()

# Analyze Learning rate: decay

In [ ]:
def build_lr(x_train):
    model=Sequential()
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(pd.DataFrame(x_train).shape[1], 1)))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    model.compile(optimizer = Adam(learning_rate=0.001) , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    return model

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],[])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)

In [ ]:
lr_model = build_lr(x_train_2d)
lr_history=lr_model.fit(x_train, y_train, batch_size=64, epochs=600, validation_data=(x_val, y_val))

In [ ]:
json.dump(lr_history.history,open("/kaggle/working/lr_history",'w'))

In [ ]:
def build_lower_lr(x_train):
    model=Sequential()
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(pd.DataFrame(x_train).shape[1], 1)))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    model.compile(optimizer = Adam(learning_rate=0.0001) , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    return model

In [ ]:
lower_lr_model = build_lower_lr(x_train_2d)
lower_lr_history=lower_lr_model.fit(x_train, y_train, batch_size=64, epochs=600, validation_data=(x_val, y_val))

In [ ]:
json.dump(lower_lr_history.history,open("/kaggle/working/lower_lr_history",'w'))

In [ ]:
def build_higher_lr(x_train):
    model=Sequential()
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(pd.DataFrame(x_train).shape[1], 1)))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    model.compile(optimizer = Adam(learning_rate=0.01) , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
    return model

In [ ]:
higher_lr_model = build_higher_lr(x_train_2d)
higher_lr_history=higher_lr_model.fit(x_train, y_train, batch_size=64, epochs=600, validation_data=(x_val, y_val))

In [ ]:
json.dump(higher_lr_history.history,open("/kaggle/working/higher_lr_history",'w'))

In [ ]:
lr_history = json.load(open("/kaggle/working/lr_history","r"))
lower_lr_history = json.load(open("/kaggle/working/lower_lr_history","r"))
higher_lr_history = json.load(open("/kaggle/working/higher_lr_history","r"))
plt.plot(lr_history["val_accuracy"],label="0.001")
plt.plot(lower_lr_history["val_accuracy"],label="0.0001")
plt.plot(higher_lr_history["val_accuracy"],label="0.01")
plt.legend()
plt.savefig("lr_acc.png")
plt.show()

In [ ]:
plt.plot(lr_history["val_loss"],label="0.001")
plt.plot(lower_lr_history["val_loss"],label="0.0001")
plt.plot(higher_lr_history["val_loss"],label="0.01")
plt.legend()
plt.savefig("lr_loss.png")
plt.show()

In [ ]:
lr_test = lr_model.evaluate(x_test,y_test)
lower_lr_test = lower_lr_model.evaluate(x_test,y_test)
higher_lr_test = higher_lr_model.evaluate(x_test,y_test)

In [ ]:
from prettytable import PrettyTable 
lr_result = PrettyTable(["learning rate","loss","accuracy"]) 
lr_result.add_row(["0.01",higher_lr_test[0],higher_lr_test[1]]) 
lr_result.add_row(["0.001",lr_test[0],lr_test[1]]) 
lr_result.add_row(["0.0001",lower_lr_test[0],lower_lr_test[1]])   
print(lr_result)

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],["noise"])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)

In [ ]:
def build_model_with_decay(x_train,initial_rate,decay_step, decay_rate):
    model=Sequential()
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(pd.DataFrame(x_train).shape[1], 1)))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_rate,
    decay_steps=decay_step,
    decay_rate=decay_rate,
    staircase=True)


    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])

    return model

In [ ]:
def compared_decay_rate():
    initial = 0.001
    steps = [27,50,70,100]
    rates = [0.96]
    historys =[]
    for step in steps:
        for rate in rates:
            model = build_model_with_decay(x_train_2d,0.001,step,rate)
            history=model.fit(x_train, y_train, batch_size=64, epochs=400, validation_data=(x_val, y_val))
            historys.append(history)
            loss_history = pd.DataFrame(history.history)
            plt.plot(loss_history['val_accuracy'],label=str(step))
    plt.legend()
    
    plt.savefig("decay_acc.png")
    plt.show()
    return historys

In [ ]:
decay_historys = compared_decay_rate()

In [ ]:
steps = [27,50,70,100]
for i in range(len(decay_historys)):
    history = decay_historys[i]
    loss_history = pd.DataFrame(history.history)
    plt.plot(loss_history['val_loss'],label=steps[i])
plt.legend()
plt.ylim((0,5))
plt.savefig("decay_loss.png")
plt.show()

# Analyze Learning rate: reduction

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],["noise"])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)

In [ ]:
def build_model_with_reduction(x_train):
    model=Sequential()
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(pd.DataFrame(x_train).shape[1], 1)))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(8))
    model.add(Activation('softmax'))



    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

    return model

In [ ]:
def compared_reduction_rate():
    factors = [0.2,0.5,0.8]
    patiences = [5,10,20]
    historys =[]
    for factor in factors:
        for patience in patiences:
            reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=factor, patience=patience, verbose=0,
            mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.0001,
            )
            callback_list = [reduce_lr]
            model = build_model_with_reduction(x_train_2d)
            history=model.fit(x_train, y_train, batch_size=64, epochs=400, validation_data=(x_val, y_val),callbacks=callback_list)
            historys.append(history)
            loss_history = pd.DataFrame(history.history)
            plt.plot(loss_history['val_accuracy'],label="factor: "+str(factor)+" patience: "+str(patience))
    plt.legend()
    
    plt.savefig("reduction_acc.png")
    plt.show()
    return historys

In [ ]:
reduction_historys = compared_reduction_rate()

In [ ]:
factors = [0.2,0.5,0.8]
patiences = [5,10,20]
for i in range(len(factors)):
    for j in range(len(patiences)):
        history = reduction_historys[i*3+j]
        loss_history = pd.DataFrame(history.history)
        plt.plot(loss_history['val_loss'],label="factor: "+str(factors[i])+" patience: "+str(patiences[j]))
plt.legend()
plt.ylim((0,5))
plt.savefig("reduction_loss.png")
plt.show()

In [ ]:
model = build_model_with_decay(x_train_2d,0.001,70,0.96)
decay_history=model.fit(x_train, y_train, batch_size=64, epochs=400, validation_data=(x_val, y_val))



In [ ]:
lower_history = lower_lr_history
#decay_history = pd.DataFrame(decay_history.history)
reduction = reduction_historys[6]
reduction_history = pd.DataFrame(reduction.history)
plt.plot(lower_history['val_accuracy'],label="0.0001")
plt.plot(decay_history['val_accuracy'],label="decay")
plt.plot(reduction_history['val_accuracy'],label="reduction")
plt.legend()
plt.savefig("lr_final_acc.png")
plt.show()

In [ ]:
plt.plot(lower_history['val_loss'],label="0.0001")
plt.plot(decay_history['val_loss'],label="decay")
plt.plot(reduction_history['val_loss'],label="reduction")
plt.legend()
plt.savefig("lr_final_loss.png")
plt.show()

# Final Model

In [ ]:
x_train_2d, y_train, x_val_2d, y_val, x_test_2d, y_test = get_features(["mfcc"],["noise"])
encoder = OneHotEncoder()
y_train = encoder.fit_transform(np.array(y_train).reshape(-1,1)).toarray()
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1)).toarray()
y_test = encoder.fit_transform(np.array(y_test).reshape(-1,1)).toarray()
x_train = np.expand_dims(x_train_2d, axis=2)
x_test = np.expand_dims(x_test_2d, axis=2)
x_val = np.expand_dims(x_val_2d, axis=2)

In [45]:
def build_model_with_decay(x_train,initial_rate,decay_step, decay_rate):
    model=Sequential()
    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu', input_shape=(pd.DataFrame(x_train).shape[1], 1)))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(256,activation="relu"))
    model.add(Dense(128,activation="relu"))
    model.add(Dense(8))
    model.add(Activation('softmax'))
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_rate,
    decay_steps=decay_step,
    decay_rate=decay_rate,
    staircase=True)


    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])

    return model

In [47]:

model = build_model_with_decay(x_train_2d,0.001,70,0.96)
history=model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_val, y_val),callbacks=callback_list)

In [48]:
model.evaluate(x_test,y_test)

In [ ]:
loss_history = pd.DataFrame(history.history)
loss_history[["accuracy","val_accuracy"]].plot()
plt.savefig("final_acc.png")

In [ ]:
loss_history[["loss","val_loss"]].plot()
plt.ylim((0,5))
plt.savefig("final_loss.png")